## Data Cleaner Script for WGCNA Application

### Install Packages and Modules

In [1]:
%%sh
pip install gffutils # https://pythonhosted.org/pyfaidx/
pip install pyfaidx # https://pythonhosted.org/gffutils/contents.html
pip install biopython


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


#### Import modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import gffutils
import pyfaidx

#### Python Analysis

In [3]:
# Notebook specific behavior setup
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = 10

%matplotlib inline 
%config InlineBackend.figure_format = 'svg'
plt.style.use("ggplot")

#### Convert xlsx to csv

In [4]:
read_file = pd.read_excel (r'NC_Pulp.xlsx')
read_file.to_csv (r'NC_Pulp.csv', index = None, header=True)

#### Load Data

In [5]:
data = "NC_Pulp.csv"

In [6]:
#I'm creating a first general dataframe common to every strategy in order to clean it and perfor some check, before creating every single df for each strategy
df = pd.read_csv(data)
df

,Gene_ID,BM2_LSI1,BM2_LSI2,BM2_LIS3,BM2_HSI1,...,M_HSI2,M_HSI3,M_LSI1,M_LSI2,M_LSI3
0,STRG.8896,0.000000,2.424661,18.123150,0.000000,...,0.000000,3.396737,0.000000,0.000000,0.000000
1,Vitvi00g00441,0.000000,0.000000,0.000000,0.000000,...,0.562196,0.849184,1.910041,0.000000,0.000000
2,Vitvi00g00443,0.000000,0.000000,12.686205,4.741684,...,0.000000,5.944290,0.000000,0.000000,0.000000
3,Vitvi00g00444,0.000000,9.698646,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
4,Vitvi00g00494,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
25351,Vitvi19g00549,78041.963262,89513.653022,68356.897749,110959.197195,...,4330.033584,4921.871772,7376.580226,4247.552403,4454.897746
25352,Vitvi02g00310,79101.432222,74857.382513,79098.488845,75641.238647,...,31427.880733,11156.582356,26381.493030,20050.024805,16923.448476
25353,Vitvi06g00321,84708.661227,77222.235690,94196.885238,101460.655956,...,21380.313840,17411.673362,25573.545481,19226.496076,20201.928688
25354,Vitvi18g02973,97320.389895,92345.657646,100425.811946,107245.510345,...,123820.295591,56147.211813,131561.747641,91980.039744,70708.594323


In [7]:
# Check for Not Null elements in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25356 entries, 0 to 25355
Data columns (total 34 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gene_ID   25356 non-null  object 
 1   BM2_LSI1  25356 non-null  float64
 2   BM2_LSI2  25356 non-null  float64
 3   BM2_LIS3  25356 non-null  float64
 4   BM2_HSI1  25356 non-null  float64
 5   BM2_HSI2  25356 non-null  float64
 6   BM3_LSI1  25356 non-null  float64
 7   BM3_LSI2  25356 non-null  float64
 8   BM3_HSI2  25356 non-null  float64
 9   BM3_HSI3  25356 non-null  float64
 10  R1_LSI1   25356 non-null  float64
 11  R1_LSI2   25356 non-null  float64
 12  R1_LSI3   25356 non-null  float64
 13  R1_HSI1   25356 non-null  float64
 14  R1_HSI2   25356 non-null  float64
 15  R1_HSI3   25356 non-null  float64
 16  R2_LSI1   25356 non-null  float64
 17  R2_LSI2   25356 non-null  float64
 18  R2_LSI3   25356 non-null  float64
 19  R2_HSI1   25356 non-null  float64
 20  R2_HSI2   25356 non-null  fl

In [8]:
#rename the first column
df.rename(columns = {'Unnamed: 0':'Gene ID'}, inplace = True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25356 entries, 0 to 25355
Data columns (total 34 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gene_ID   25356 non-null  object 
 1   BM2_LSI1  25356 non-null  float64
 2   BM2_LSI2  25356 non-null  float64
 3   BM2_LIS3  25356 non-null  float64
 4   BM2_HSI1  25356 non-null  float64
 5   BM2_HSI2  25356 non-null  float64
 6   BM3_LSI1  25356 non-null  float64
 7   BM3_LSI2  25356 non-null  float64
 8   BM3_HSI2  25356 non-null  float64
 9   BM3_HSI3  25356 non-null  float64
 10  R1_LSI1   25356 non-null  float64
 11  R1_LSI2   25356 non-null  float64
 12  R1_LSI3   25356 non-null  float64
 13  R1_HSI1   25356 non-null  float64
 14  R1_HSI2   25356 non-null  float64
 15  R1_HSI3   25356 non-null  float64
 16  R2_LSI1   25356 non-null  float64
 17  R2_LSI2   25356 non-null  float64
 18  R2_LSI3   25356 non-null  float64
 19  R2_HSI1   25356 non-null  float64
 20  R2_HSI2   25356 non-null  fl

,Gene_ID,BM2_LSI1,BM2_LSI2,BM2_LIS3,BM2_HSI1,...,M_HSI2,M_HSI3,M_LSI1,M_LSI2,M_LSI3
0,STRG.8896,0.000000,2.424661,18.123150,0.000000,...,0.000000,3.396737,0.000000,0.000000,0.000000
1,Vitvi00g00441,0.000000,0.000000,0.000000,0.000000,...,0.562196,0.849184,1.910041,0.000000,0.000000
2,Vitvi00g00443,0.000000,0.000000,12.686205,4.741684,...,0.000000,5.944290,0.000000,0.000000,0.000000
3,Vitvi00g00444,0.000000,9.698646,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
4,Vitvi00g00494,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
25351,Vitvi19g00549,78041.963262,89513.653022,68356.897749,110959.197195,...,4330.033584,4921.871772,7376.580226,4247.552403,4454.897746
25352,Vitvi02g00310,79101.432222,74857.382513,79098.488845,75641.238647,...,31427.880733,11156.582356,26381.493030,20050.024805,16923.448476
25353,Vitvi06g00321,84708.661227,77222.235690,94196.885238,101460.655956,...,21380.313840,17411.673362,25573.545481,19226.496076,20201.928688
25354,Vitvi18g02973,97320.389895,92345.657646,100425.811946,107245.510345,...,123820.295591,56147.211813,131561.747641,91980.039744,70708.594323


In [ ]:
#reorder columns
df = pd.DataFrame(df[['Gene ID','BM2_LSI1','BM2_LSI2','BM2_LSI3','BM2_HSI1','BM2_HSI2','BM2_HSI3', 'BM3_LSI1','BM3_LSI2','BM3_LSI3','BM3_HSI1','BM3_HSI2','BM3_HSI3', 'R1_LSI1', 'R1_LSI2', 'R1_LSI3', 'R1_HSI1', 'R1_HSI2', 'R1_HSI3', 'M_HSI1', 'M_HSI2', 'M_HSI3', 'M_LSI1', 'M_LSI2', 'M_LSI3', 'R2_LSI1', 'R2_LSI2', 'R2_LSI3', 'R2_HSI1', 'R2_HSI2', 'PR_HSI1', 'PR_HSI2', 'PR_HSI3', 'PR_HSI4', 'PR_LSI1', 'PR_LSI2']])
df.info()





In [9]:
df_passed = df
df_declined = df

for r in range(df.shape[0]):
    count = 0
    for i in range(10, 34, 6):
        if ((df.iloc[r, i] != 0 and df.iloc[r, i+1] != 0 and df.iloc[r, i+2] != 0 and df.iloc[r, i+3] != 0 and df.iloc[r, i+4] != 0 and df.iloc[r, i+5] != 0) and ((df.iloc[r, i] + df.iloc[r, i+1] + df.iloc[r, i+2]) > 4.70043972) and ((df.iloc[r, i+3] + df.iloc[r, i+4] + df.iloc[r, i+5]) > 4.70043972)):
            count += 1
    for i in range(1, 2):
        if ((df.iloc[r, i] != 0 and df.iloc[r, i+1] != 0 and df.iloc[r, i+2] != 0 and df.iloc[r, i+3] != 0 and df.iloc[r, i+4] != 0) and ((df.iloc[r, i] + df.iloc[r, i+1] + df.iloc[r, i+2]) > 4.70043972) and ((df.iloc[r, i+3] + df.iloc[r, i+4]) > 4.70043972)):
            count += 1
    for i in range(6, 7):
        if ((df.iloc[r, i] != 0 and df.iloc[r, i+1] != 0 and df.iloc[r, i+2] != 0 and df.iloc[r, i+3] != 0) and ((df.iloc[r, i] + df.iloc[r, i+1]) > 4.70043972) and ((df.iloc[r, i+2] + df.iloc[r, i+3]) > 4.70043972)):
            count += 1
            
    if count != 6:
        df_passed = df_passed.drop(df.index[r])
    if count == 6:
        df_declined = df_declined.drop(df.index[r])

        
df_passed.shape
df_declined.shape
df.shape

(13236, 34)

(12120, 34)

(25356, 34)

In [11]:
df_passed.head()
df_passed.eq(0).any().any()
df_declined.eq(0).any().any()

,Gene_ID,BM2_LSI1,BM2_LSI2,BM2_LIS3,BM2_HSI1,...,M_HSI2,M_HSI3,M_LSI1,M_LSI2,M_LSI3
5872,Vitvi00g01341,1.395875,0.808220,5.436945,5.690021,...,2.248784,1.698368,1.910041,4.639598,5.531744
5904,Vitvi03g00943,1.395875,4.041102,9.061575,7.586694,...,8.432940,4.245921,7.640166,4.639598,11.063488
5958,Vitvi07g03014,1.395875,7.273984,5.436945,4.741684,...,5.059764,5.944290,5.730124,11.598996,3.687829
6065,Vitvi17g00606,1.395875,6.465764,7.249260,8.535031,...,10.119528,15.285316,5.730124,6.959398,3.687829
6095,Vitvi00g00974,2.791750,4.041102,18.123150,10.431705,...,12.368312,5.095105,5.730124,13.918795,5.531744


False

True

In [12]:
outputfile= 'NC_Pulp_Passed.csv'
df_passed.to_csv(outputfile,index=False)

outputfile= 'NC_Pulp_Declined.csv'
df_declined.to_csv(outputfile,index=False)

In [ ]:
read_file = pd.read_csv (r'NC_Pulp_Passed.csv')
read_file.to_excel (r'NC_Pulp_Passed.xlsx', index = None, header=True)

read_file = pd.read_csv (r'NC_Pulp_Declined.csv')
read_file.to_excel (r'NC_Pulp_Declined.xlsx', index = None, header=True)

*Notebook Created By: Christian Mandelli, Oregon State University*